In [1]:
!rm ag_news_csv.tgz
!wget https://s3.amazonaws.com/fast-ai-nlp/ag_news_csv.tgz
!tar -xvzf ag_news_csv.tgz

--2019-10-05 19:54:58--  https://s3.amazonaws.com/fast-ai-nlp/ag_news_csv.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.108.165
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.108.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11784419 (11M) [application/x-tar]
Saving to: ‘ag_news_csv.tgz’

ag_news_csv.tgz     100%[===================>]  11.24M  60.2MB/s    in 0.2s    

2019-10-05 19:54:58 (60.2 MB/s) - ‘ag_news_csv.tgz’ saved [11784419/11784419]

ag_news_csv/
ag_news_csv/train.csv
ag_news_csv/readme.txt
ag_news_csv/test.csv
ag_news_csv/classes.txt


In [2]:
import pandas as pd
from tqdm import tqdm_notebook

df = {}
keys = ['train', 'test']
for k in keys :
    df[k] = pd.read_csv('ag_news_csv/' + k + '.csv', header=None)

classes = open('ag_news_csv/classes.txt').read().strip().split('\n')
for k in keys :
    df[k][0] = df[k][0].apply(lambda x : classes[x-1])

import pandas as pd
df_texts = []
df_labels = []
df_exp_splits = []

for key in ['train', 'test'] :
    df_texts += list(df[key][2])
    df_labels += list(df[key][0])
    df_exp_splits += [key] * len(list(df[key][2]))
    
df = pd.DataFrame({'text' : df_texts, 'label' : df_labels, 'exp_split' : df_exp_splits})

from sklearn.model_selection import train_test_split
train_idx, dev_idx = train_test_split(df.index[df.exp_split == 'train'], test_size=0.15, random_state=16377)
df.loc[dev_idx, 'exp_split'] = 'dev'
df.to_csv('agnews_dataset_split_full.csv', index=False)

In [3]:
import pandas as pd
import json

import os
os.makedirs('data', exist_ok=True)

In [4]:
data = pd.read_csv('agnews_dataset_split_full.csv')

In [5]:
for key in ['train', 'dev', 'test'] :
    data_key = data[data.exp_split == key].to_dict(orient='records')
    data_key = [{'document' : x['text'], 'label' : x['label']} for x in data_key]
    f = open('data/' + key + '.jsonl', 'w')
    f.write('\n'.join([json.dumps(line) for line in data_key]))
    f.close()